# Preparing the data

In [ ]:
# Prepare the toolbox

import pandas as pd
import matplotlib.pyplot as plt
import random
import os

In [ ]:
# Create a single DataFrame from the different excel files

df = pd.DataFrame()

data_dir = "./data/raw/"

for file in os.listdir(data_dir):
    if file.endswith(".xlsx"):
        data = pd.read_excel(data_dir + file)
        df = pd.concat([df, data])
        
df = df.drop_duplicates().reset_index(drop=True)
df

In [ ]:
# Check for NaN values

any(df.isna().sum())

In [ ]:
# Check that types are correct

print(df.info(verbose=True))

In [ ]:
# Clean strings

# The method str provides an access. 
# Without str, strip returns an error, because it would be acting on a Series instead of a string.
df["ANIMAL"] = df["ANIMAL"].str.strip("'")
df["STATION"] = df["STATION"].str.strip("'")
df["DIVISION"] = df["DIVISION"].str.strip("'")
df["DIRECTION"] = df["DIRECTION"].str.strip("'")

In [ ]:
# Take only IC neurons: these have more stereotypical responses than those in the MGB and AC. 
# We can reincorporate MGB and AC neurons in a later stage of the project.

df_IC = df[df["STATION"]=='IC'].copy() # IMPORTANT: copy() to avoid that changes made in df_IC propagate back to df!
df_IC.reset_index(drop=True, inplace=True)
df_IC

In [ ]:
# Convert to strings the columns providing ID information

df_ids = df_IC[df_IC.columns[0:8].to_list()].applymap(str)
df_ids.info()

In [ ]:
# Create unique identifiers for each PSTH vector while cleaning information we won't need for the clustering

df_IC['id'] = ""

for i in range(len(df_IC)):
    df_IC.loc[i,'id'] = "/".join(df_ids.iloc[i]) # We can get the information back later by using the split method
    
df_IC.drop(df_ids.columns, axis=1, inplace=True)

In [ ]:
# Check if there are any duplicates in ID

any(x == True for x in df_IC['id'].duplicated())

In [ ]:
# Some measurements with the same experimental conditions have been repeated, resulting in duplicated IDs. Which ones?

df_IC[df_IC['id'].duplicated()]

In [ ]:
# Handle duplicated IDs

for i in df_IC[df_IC['id'].duplicated()].index:
    df_IC.loc[i,'id'] = df_IC.loc[i,'id']+'_2' # Remember to remove this _2 if the ID is split("/") back
    
any(x == True for x in df_IC['id'].duplicated()) # Check if there are any duplicated IDs now

In [ ]:
# Save DataFrame to a csv
df_IC.to_csv("./data/psth_data_IC.csv",index=False)

In [ ]:
# Load DataFrame to check that everything is OK

data = pd.read_csv("./data/psth_data_IC.csv")
data

In [ ]:
# Check one example to see that everything is OK

data.iloc[random.randrange(0,len(data.index)-1)].drop('id').plot(kind='line')
plt.xlabel('Time (ms)')
plt.ylabel('Spike density (norm)')
plt.show()